In [9]:
!nvidia-smi

Tue Dec 20 10:19:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    25W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
# 상용 가능?
 
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import urllib.request
import pandas as pd

In [12]:
from sentence_transformers import SentenceTransformer

In [14]:
# urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('./bash_dataset.csv')
train_data.tail()

,Q,A
105,i need datasets shown in details,listcat -a
106,list all catalog information in details,listcat -a
107,i wanna see all catalog information in specific,listcat -a
108,dump osc sd information from OSCOIVP1 into dat...,oscsddump -r OSCOIVP1 data.dump
109,save sd data of region TEST5 as sddata.txt,oscsddump -r TEST5 sddata.txt


링크 : https://github.com/jhgan00/ko-sentence-transformers

In [15]:
# model = SentenceTransformer('jhgan/ko-sroberta-multitask')
model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')

Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.5k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/383 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [16]:
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)

In [17]:
train_data

,Q,A,embedding
0,execute regions,oscboot,"[0.11084972, 0.057418406, -0.057361893, 0.0041..."
1,start regions,oscboot,"[0.102295846, 0.008300843, -0.014352412, -0.02..."
2,boot regions,oscboot,"[0.0995663, 0.035809014, 0.020648062, 0.020353..."
3,boot engins,oscboot,"[-0.037645984, -0.0015464679, 0.025727153, -0...."
4,turn on regions,oscboot,"[0.10956384, 0.020232234, -0.00043220658, 0.03..."
...,...,...,...
105,i need datasets shown in details,listcat -a,"[-0.015067677, -0.009111492, -0.008583075, -0...."
106,list all catalog information in details,listcat -a,"[0.03191917, 0.040864892, -0.07080508, -0.0045..."
107,i wanna see all catalog information in specific,listcat -a,"[0.025197282, -0.026122244, -0.0807604, -0.014..."
108,dump osc sd information from OSCOIVP1 into dat...,oscsddump -r OSCOIVP1 data.dump,"[-0.041970026, 0.057105064, -0.055600446, 0.02..."


In [18]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [19]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [20]:
def return_similar_answer(input):
    embedding = model.encode(input)
    train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    return train_data.loc[train_data['score'].idxmax()]['A']

In [21]:
return_similar_answer('i need to boot regions without tmax engines only')

'oscboot -C'

In [22]:
return_similar_answer('it seems that the database is unnecessary')

'tbdown'

In [23]:
return_similar_answer("i need to see the datasets in datails")

'listcat -a'

In [24]:
return_similar_answer("i am in need of db")

'tbboot'